## data integration

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from functools import partial
from shapely import ops
import pyproj
import json, math
import matplotlib.pyplot as plt
from folium import plugins
import re

from matplotlib import font_manager, rc, rcParams
# def set_korea_font():
#     font_name = font_manager.FontProperties(fname="/System/Library/Fonts/Supplemental/AppleGothic.ttf").get_name()
#     rc('font', family=font_name)
#     rcParams.update({'font.size': 11})
#     rcParams['axes.unicode_minus'] = False  
# set_korea_font()

from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi

from geopy.distance import geodesic
import folium
from folium import plugins
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import os
import datetime
os.chdir('./osan/data')

### 홍익선
1. 과속단속 카메라 개수
2. 유동인구 평균
3. 법규위반
4. 과속방지턱
5. 불법주정차
    - cctv 1개당 평균 불법주정차 단속 건수
    
### 조민주
1. 차량등록현황
2. 어린이 거주인구
	- 어린이 거주 비율
3. 주변 학교/어린이집 개수
4. 스쿨존 여부
5. 스쿨존 중복 포함 개수

### 심재훈
1. 사고 개수
2. 혼잡빈도
3. 너비
4. 교차로

### 이재훈
1. 학원 개수
2. 횡단보도 갯수
3. 신호등 갯수
4. 인도 여부

### data import

In [2]:
#tab에서 찾기 편하게 모든 단어에 _count_ 붙음
#refer_data 는 data_set['function name']
#홍익선
overspeed_cam_count_ = pd.read_csv('overspeed_cam.csv')
floating_pop_count_ = pd.read_csv('floating_pop.csv')
child_pedestrian_count_ = pd.read_csv('child_pedestrian.csv')
bump_count_ = pd.read_csv('bump.csv')
parking_count_ = pd.read_csv('parking.csv')
parking_cctv_count_ = [parking_count_, pd.read_csv('parking_cctv.csv')]

#조민주
car_count_ = gpd.read_file("3.오산시_차량등록현황_격자.geojson", encoding = "utf-8")
child_count_ = gpd.read_file("child_count.geojson", encoding = "utf-8")
child_rate_count_ = gpd.read_file("child_rate.geojson", encoding = "utf-8")
elem_kinder_count_ = gpd.read_file("elem_kinder_count.csv", encoding = "utf-8")
isSchoolZone_count_ = pd.read_csv("9.오산시_어린이보호구역(re).csv", encoding = "utf-8")
numberSchoolZone_count_ = isSchoolZone_count_

#심재훈
accident_count_ = pd.read_csv("accident_count.csv")
chaos1_nearby_count_ = [gpd.read_file('23.오산시_상세도로망_LV6.geojson'),
                 pd.read_csv('25.평일_전일_오산시_혼잡빈도강도_Level6.csv')]
width_nearby_count_ = gpd.read_file('23.오산시_상세도로망_LV6.geojson')
cross_road_nearby_count_ = width_nearby_count_

#이재훈
num_cram_school_count_ = pd.read_csv("학원.csv")
shortest_cross_count_ = gpd.read_file("횡단보도.json",encoding="utf-8")
shortest_traffic_signal_count_ = gpd.read_file('19.오산시_신호등.geojson',encoding="utf-8")
shortest_sidewalk_count_ =  gpd.read_file("인도.json",encoding="utf-8")


data_set = {
'overspeed_cam_count': overspeed_cam_count_,
'floating_pop_count':floating_pop_count_,
'child_pedestrian_count':child_pedestrian_count_,
'bump_count':bump_count_,
'parking_count':parking_count_,
'parking_cctv_count':parking_cctv_count_,
'car_count':car_count_,
'child_count':child_count_,
'child_rate_count':child_rate_count_,
'elem_kinder_count':elem_kinder_count_,
'isSchoolZone_count':isSchoolZone_count_,
'numberSchoolZone_count':numberSchoolZone_count_,
'accident_count':accident_count_,
'chaos1_nearby_count':chaos1_nearby_count_,
'width_nearby_count':width_nearby_count_,
'cross_road_nearby_count':cross_road_nearby_count_,
'num_cram_school_count':num_cram_school_count_,
'shortest_cross_count':shortest_cross_count_,
'shortest_traffic_signal_count':shortest_traffic_signal_count_,
'shortest_sidewalk_count':shortest_sidewalk_count_,
}

In [3]:
data_set = {
'overspeed_cam_count': overspeed_cam_count_,
'floating_pop_count':floating_pop_count_,
'child_pedestrian_count':child_pedestrian_count_,
'bump_count':bump_count_,
'parking_count':parking_count_,
'parking_cctv_count':parking_cctv_count_,
'car_count':car_count_,
'child_count':child_count_,
'child_rate_count':child_rate_count_,
'elem_kinder_count':elem_kinder_count_,
'isSchoolZone_count':isSchoolZone_count_,
'numberSchoolZone_count':numberSchoolZone_count_,
'accident_count':accident_count_,
'chaos1_nearby_count':chaos1_nearby_count_,
'width_nearby_count':width_nearby_count_,
'cross_road_nearby_count':cross_road_nearby_count_,
'num_cram_school_count':num_cram_school_count_,
'shortest_cross_count':shortest_cross_count_,
'shortest_traffic_signal_count':shortest_traffic_signal_count_,
'shortest_sidewalk_count':shortest_sidewalk_count_,
}

## 홍익선

## 1. 과속단속 카메라 개수

In [4]:
def overspeed_cam_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m)
  output: following point count
  optional: (idx of point, count)
  """

  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

## 2. 유동인구 평균

In [5]:
def floating_pop_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """

  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon), x).m <= radi)
  if idx:
    return refer_data[bool_mask].index, refer_data['mean'][bool_mask].mean()
  else:
    return refer_data['mean'][bool_mask].mean()

## 3. 법규위반

In [6]:
#어린이 보행/자전거 전체 사고 횟수
def child_pedestrian_count(lat, lon, radi, refer_data, typ = 'all', idx=False):
  """
  input: origin lat lon, radi(m), typ
  typ = 'all', '안전운전불이행','보행자보호의무위반', '신호위반'
  
  output: following point count
  optional: (idx of point, count)
  """

  #apply
  if typ != 'all':
    refer_data = refer_data.loc[refer_data.법규위반==typ]
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

## 4. 과속방지턱

In [7]:
def bump_count(lat, lon, radi, refer_data, idx = False):
    dist = refer_data[['index','geometry']].apply(lambda x: (geodesic((lat, lon), x[1]).m, x[0]), axis = 1)
    dist = min(dist, key = lambda x :x[0])
                                        
    if idx:
        return dist
    else:
        return dist[0]

## 5. 불법주정차

In [8]:
def parking_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """

  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

## 5-1. cctv 1개당 평균 불법주정차 단속 건수

In [9]:
def parking_cctv_count(lat, lon, radi, refer_data, idx=False):
  """
  cctv 1개당 평균 불법주정차 단속 건수
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """
  refer_data1, refer_data2 = refer_data[0], refer_data[1]
  #apply
  bool_mask1 = refer_data1['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  bool_mask2 = refer_data2['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)

  if idx:
    return refer_data1[bool_mask1].index, refer_data2[bool_mask2].index, sum(bool_mask1)/sum(bool_mask2)
  else:
    return sum(bool_mask1)/sum(bool_mask2)

## 조민주

In [10]:
osan = (37.1623799231016, 127.05436890115905)

In [11]:
#from https://medium.com/analytics-vidhya/calculating-distances-from-points-to-polygon-borders-in-python-a-paris-example-3b597e1ea291
def midpoint(a, b):
    a_lat, a_lon = radians(a.latitude), radians(a.longitude)
    b_lat, b_lon = radians(b.latitude), radians(b.longitude)
    delta_lon = b_lon - a_lon
    B_x = cos(b_lat) * cos(delta_lon)
    B_y = cos(b_lat) * sin(delta_lon)
    mid_lat = atan2(
        sin(a_lat) + sin(b_lat),
        sqrt(((cos(a_lat) + B_x) ** 2 + B_y ** 2))
    )
    mid_lon = a_lon + atan2(B_y, cos(a_lat) + B_x)
    # Normalise
    mid_lon = (mid_lon + 3 * pi) % (2 * pi) - pi

    return Point(latitude=degrees(mid_lat), longitude=degrees(mid_lon))

def get_line_midpoint(line):
    a = Point(line[0])
    b = Point(line[1])

    return midpoint(a,b)


def calculate_dist_to_line(line_a_lat, line_a_lng, line_b_lat, line_b_lng, point_object):
    a = Point(latitude=line_a_lat, longitude=line_a_lng)
    b = Point(latitude=line_b_lat, longitude=line_b_lng)
    dist = distance(midpoint(a, b), point_object)
    return dist


def get_min_distance_to_arr(arr_coords, point_object, unit='m'):
    min_dist = 999999
    line=[]
    for i, _ in enumerate(arr_coords):
        if i + 1 < len(arr_coords):
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[i + 1][1],
                line_b_lng=arr_coords[i + 1][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[i + 1][1], arr_coords[i + 1][0])]
        else:
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[0][1],
                line_b_lng=arr_coords[0][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[0][1], arr_coords[0][0])]

    if unit == 'm':
        return min_dist.m, line
    elif unit == 'km':
        return min_dist.km, line
    else:
        return min_dist, line
    
#additional interrial function
def _multiline_to_arr_cords(mls):
  """
  input: multilinestring object
  output: arr_cords; lon, lat
  ex) multiline_to_arr_cords(road_gdf.loc[0].geometry)
  """
  arr_cords = []
  lon_lat = re.findall(r'\d+\.\d+',str(mls))
  for i in range(0,len(lon_lat),2):
    arr_cords.append((float(lon_lat[i]),float(lon_lat[i+1])))
  return arr_cords

#(lat, lon), (lon, lat) 자꾸 갈려서 나오네
def _shPoint_to_list(shpoint):
  c1, c2 = re.findall(r'\d+\.\d+',str(shpoint))
  if c1 < c2:
    lon, lat = float(c1), float(c2)
  else:
    lat, lon = float(c1), float(c2)
  return lat, lon

def _list_to_gpPoint(mylist):
  return Point(latitude=mylist[0], longitude=mylist[1])

def _shPoint_to_gpPoint(shpoint):
  mylist = _shPoint_to_list(shpoint)
  return Point(latitude=mylist[0], longitude=mylist[1])

In [12]:
def road_nearby(lat, lon, radi, data):
  """
  input: origin lat lon, radi(m), data(shapely multilinestring)
  change in type: shapely(Euclidian) -> list -> geopy(Great Circle)
  #watchout for (lat lon), (lon lat)
  output: following roads
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  road_nearby(lat, lon, radi, road_gdf)
  """
  road_gdf = data
  dist_array = road_gdf.geometry.apply(lambda x: get_min_distance_to_arr(_multiline_to_arr_cords(x), _shPoint_to_gpPoint((lat, lon)), unit='m')[0])
  return road_gdf.loc[dist_array < radi,:]

## 2. 차랑등록현황

In [13]:
def car_count(lat, lon, radi, refer_data):
    """
    default unit = 'm'
    """   
    distance_arr = data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    sum_count = sum(refer_data[distance_arr <= radi]["car_cnt"])
    return sum_count

## 3. 어린이 거주인구 

In [14]:
def child_count(lat, lon, radi, refer_data):
    refer_data.val[data.val.isna()] = 0
    distance_arr = data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    sum_count = sum(refer_data[distance_arr <= radi]["val"])
    return sum_count

## 3.5 어린이 거주 비율 

In [15]:
def child_rate_count(lat, lon, radi, refer_data):
    distance_arr = data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    selected_data = refer_data[distance_arr <= radi]
    child_rate = sum(selected_data["val"])/sum(selected_data["total"])
    return child_rate

## 4. 주변 학교/어린이집 개수

In [16]:
def elem_kinder_count(lat, lon, radi, refer_data, typ = 'all'):
    """
    typ = 'all', '유치원과 어린이집', '초등학교'
    """
    if typ != 'all':
        refer_data = refer_data.loc[refer_data.시설구분==typ]
    data_loc = pd.Series(list(zip(refer_data["시설위치_위도"], refer_data["시설위치_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= radi).sum()
    return count_sum

## 5. 스쿨존 여부 : isSchoolZone(lat, lon, radi, refer_data)
### 하지만 정확한 스쿨존 범위를 모름

In [17]:
def isSchoolZone_count(lat, lon, radi, refer_data): # radi = 300 이지만 순서 통일을 위해 default값 지정 x
    data_loc = pd.Series(list(zip(refer_data["보호구역_위도"], refer_data["보호구역_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= 300).sum()
    return count_sum > 0

## 5.5 몇 개의 스쿨존에 포함되는지 : numberSchoolZone(lat, lon, radi, refer_data)
### CCTV 배치에 도움이 될 수도..?

In [18]:
def numberSchoolZone_count(lat, lon, radi, refer_data): # radi = 300 이지만 순서 통일을 위해 default값 지정 x
    data_loc = pd.Series(list(zip(refer_data["보호구역_위도"], refer_data["보호구역_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= 300).sum()
    return count_sum

## 심재훈

### accident_count

In [19]:
def accident_count(lat, lon, radi, data, idx=False):
  """
  input: origin lat lon, radi(m), data(accidents record)
  output: following point count
  optional: (idx of point, count)
  """
  crawled_df = data

  #apply
  bool_mask = crawled_df['geometry'].apply(lambda x: geodesic((lat, lon),x).m <= radi)
  if idx:
    return crawled_df[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

In [20]:
def chaos1_nearby_count(lat, lon, radi, refer_data):
  """
  input: origin lat lon, radi(m), road_data(shapely multilinestring), chaos1_data
  output: following road's chaos1 by different times or lanes
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  chaos1_nearby(lat, lon, radi, road_gdf, chaos1_gdf)[11]
  #link_id 끝자리 01 02 순으로 나옴
  """
  road_data, chaos1_data = refer_data[0], refer_data[1]
  roads = road_nearby(lat, lon, radi, road_data)
  result = roads.link_id.apply(lambda x: chaos1_data.loc[(chaos1_data.loc[:,'link_id']//100)==int(x),['time_range','work/weekend','chaos1']])
  return result

def width_nearby_count(lat, lon, radi, road_data):
  roads = road_nearby(lat, lon, radi, road_data)
  return roads.width

def cross_road_nearby_count(lat, lon, radi, road_data):
  """
  4	교차로 통로
  32	복합교차로
  64	로타리
  128	회전교차로

  output = # of cross roads
  """
  cross_roads = [4,32,64,128]
  roads = road_nearby(lat, lon, radi, road_data)
  result = roads.loc[roads.link_type.apply(lambda x: int(x) in cross_roads)]
  return len(result)

## 이재훈

### 주변 학원 개수

In [21]:
def num_cram_school_count(lat, lon, radi, refer_data):
    cram_school = pd.Series(list(zip(refer_data["시설위치_위도"], refer_data["시설위치_경도"])))
    return cram_school.apply(lambda x: geodesic((lat, lon),x).m <= radi).sum()

### 횡단보도 거리

In [22]:
#reproject 후 polygon 그대로
def shortest_cross_count(lat, lon, radi, refer_data, idx = False):
    polygon1 = reproject(Point(lon,lat), 'EPSG:4326', 'EPSG:26944')
    result = refer_data[["OBJECTID","geometry"]].apply(lambda x : (x[1].distance(polygon1), x[0]), axis = 1)
    result = min(result, key = lambda x :x[0])
    if idx:
        return result
    return result[0]

### 신호등 거리

In [23]:
def shortest_traffic_signal_count(lat, lon, radi, refer_data, idx = False):
    result = refer_data[["OBJECTID","geometry"]].apply(lambda x : (geodesic((lat, lon),(x[1].y,x[1].x)).m, x[0]), axis = 1)
    result = min(result, key = lambda x :x[0])
    if idx:
        return result
    return result[0]

### 인도 여부

In [24]:
def shortest_sidewalk_count(lat, lon, radi, refer_data):
    polygon1 = reproject(Point(lon,lat), 'EPSG:4326', 'EPSG:26944')
    result = refer_data["geometry"].apply(lambda x : x.distance(polygon1))
    result = min(result)
    return result